In [ ]:
# Corrección de cuantiles empírica.

import os

import pandas as pd
import numpy  as np

In [ ]:
# Rutas de archivos.

path_d = "../results/NSRDB/"
path_r = "../results/NSRDB_2/"
path_c = "../results/NSRDB_quantile/"

if not os.path.exists(path_c): os.mkdir(path_c)

files_d = os.listdir(path_d)
files_r = os.listdir(path_r)

if ".DS_Store" in files_d: files_d.remove(".DS_Store")
if ".DS_Store" in files_r: files_r.remove(".DS_Store")

In [ ]:
# Iteramos para todas las coordenadas.
for i in range( len(files_d) ):
    df_d = pd.read_csv( f"{path_d}{files_d[i]}", index_col = "time",
        parse_dates = True, infer_datetime_format = True )
    df_r = pd.read_csv( f"{path_r}{files_r[i]}", index_col = "time",
        parse_dates = True, infer_datetime_format = True )

    # Creamos un DataFrame de apoyo.
    df_c = df_r.copy()
    # Iteramos para todas las columnas.
    for v in df_d.columns:
        # Ordenamos los valores originales incluyendo el tiempo.
        df_i = df_d[ [v] ].sort_values(v).reset_index()
        # Calculamos la distribución acumulada.
        df_i["CDF"] = np.linspace( 1 / df_i.shape[0] , 1, df_i.shape[0] )

        # Ordenamos los valores destino incluyendo el tiempo.
        df_j = df_r[ [v] ].sort_values(v).reset_index()
        # Calculamos la distribución acumulada.
        df_j["CDF"] = np.linspace( 1 / df_j.shape[0] , 1, df_j.shape[0] )

        # Interpolamos los valores de la distribución origen a la destino.
        df_j["Cuant"] = np.interp( df_j["CDF"], df_i["CDF"],
            df_i[v] ).round( decimals = 1 )
        # Reordenamos los valores.
        df_c[v] = df_j.sort_values("time").set_index( "time" ).drop( [v, "CDF"],
            axis = 1 ).rename( {"Cuant": v}, axis = 1 )

    df_c.to_csv( f"{path_c}{files_r[i]}" )